In [1]:
!pip install torch torchvision torchaudio


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install transformers requests beautifulsoup4 numpy pandas
#transformers - used for nlp model , import download and install our nlp model ie a multilingual bert model
#requests - request to website for web scraping
#beautifulsoup4 - work throught data from the site and extract and accept data we actually need
#pandas - used to structure our data to make it easier to work with
#numpy - additional data transformation processes

In [2]:
from transformers import AutoTokenizer , AutoModelForSequenceClassification
# AutoTokenizer - preprocess text data by converting it into tokens
#AutoModelForSequenceClassification - automatically loads a pre-trained model for sequence classification tasks
import torch
import requests
from bs4 import BeautifulSoup
import re

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
#used to load a pre-trained tokenizer and model for sentiment analysis using the Hugging Face Transformers library
#model is trained to perform sentiment analysis and classify text into sentiment categories.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [9]:
tokens = tokenizer.encode('It was okay , could be better',return_tensors='pt')
#return_tensors='pt' argument ensures that the output is in the form of a PyTorch tensor.
result = model(tokens)
result.logits
#One-hot encoding - used to represent categorical data as binary vectors.

tensor([[-1.4785,  1.2036,  2.9974,  0.1951, -2.5068]],
       grad_fn=<AddmmBackward0>)

In [10]:
int(torch.argmax(result.logits)+1)

3

In [13]:
r = requests.get('https://www.yelp.com/biz/taco-bell-san-francisco-12')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [15]:
reviews

["I enjoyed Taco Bell like it was an expensive fancy meal lol not. I went in at this location at around to try the new chile verde fries and an enchirito.  The chile verde fries were as fresh as they could get. The food was even still steaming from how hot it was. I can't remember the last time I enjoyed Taco Bell like this. The neighborhood is great, and the employees working at this location were far more mature than those of other locations I have visited in the past.  Definitely my go to place in a pinch.",
 '"Taco Bell has always been my go-to spot for a quick, satisfying meal that doesn\'t break the bank. Whether it\'s a late-night craving or a quick lunch on the go, they always deliver exactly what I\'m in the mood for. The menu is packed with variety, from classic tacos and burritos to some of their more creative items like the Crunchwrap Supreme or the Doritos Locos Tacos. No matter what I order, I know it\'s going to be tasty, fresh, and just the right amount of indulgent.One

In [16]:
import pandas as pd
import numpy as np

In [18]:
dataframe = pd.DataFrame(np.array(reviews), columns=['review'])
dataframe.tail()

,review
5,Yum ba hahah blastie poo
6,"Tasty food, quick service. It's my goto place ..."
7,I love the new cheeze it Crunchwrap! So yummy!...
8,wow just wow completely devastated with my exp...
9,mario and the manager hunter were so amazing!!...


In [20]:
#iloc - allows us to select rows and columns by their integer positions in a DataFrame or Series.
dataframe['review'].iloc[0]

"I enjoyed Taco Bell like it was an expensive fancy meal lol not. I went in at this location at around to try the new chile verde fries and an enchirito.  The chile verde fries were as fresh as they could get. The food was even still steaming from how hot it was. I can't remember the last time I enjoyed Taco Bell like this. The neighborhood is great, and the employees working at this location were far more mature than those of other locations I have visited in the past.  Definitely my go to place in a pinch."

In [22]:
def sentiment(review):
    tokens = tokenizer.encode('review',return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits)+1)


In [26]:
sentiment(dataframe['review'].iloc[1])
dataframe.head()

,review
0,I enjoyed Taco Bell like it was an expensive f...
1,"""Taco Bell has always been my go-to spot for a..."
2,I was here again about 3wks ago and I'm 1000% ...
3,I asked for a Baja blast and she gave me boba...
4,Taco Bells and McDonalds..hahah KFC fried chic...


In [30]:
dataframe['sentiment'] = dataframe['review'].apply(lambda x: sentiment(x[:512]))


In [32]:
dataframe


,review,sentiment
0,I enjoyed Taco Bell like it was an expensive f...,4
1,"""Taco Bell has always been my go-to spot for a...",4
2,I was here again about 3wks ago and I'm 1000% ...,4
3,I asked for a Baja blast and she gave me boba...,4
4,Taco Bells and McDonalds..hahah KFC fried chic...,4
5,Yum ba hahah blastie poo,4
6,"Tasty food, quick service. It's my goto place ...",4
7,I love the new cheeze it Crunchwrap! So yummy!...,4
8,wow just wow completely devastated with my exp...,4
9,mario and the manager hunter were so amazing!!...,4


In [35]:
dataframe['review'].iloc[0]

"I enjoyed Taco Bell like it was an expensive fancy meal lol not. I went in at this location at around to try the new chile verde fries and an enchirito.  The chile verde fries were as fresh as they could get. The food was even still steaming from how hot it was. I can't remember the last time I enjoyed Taco Bell like this. The neighborhood is great, and the employees working at this location were far more mature than those of other locations I have visited in the past.  Definitely my go to place in a pinch."